
## 🧠 Core Concept (very important)

### Handoff is one-way.

Once control is handed off, the new agent stays in control until you explicitly route again.

The SDK does not implement an automatic “return to router” mechanism.

In [12]:
from dotenv import load_dotenv
from agents import Agent, Runner, handoff, RunContextWrapper, function_tool
from pydantic import BaseModel, Field
load_dotenv()

True

In [6]:
class Tutorial(BaseModel):
    outline: str
    tutorial: str 

tutorial_generator = Agent(
    name="Tutorial Generator",
    instructions=("Given a proframming topic and an outline, your job is to generate code snippets for each section of the outline."
                  "Format the tutorial in Markdown using a mix of text for explanaton and code snippets for examples."
                  "Where it makes sense, include comments in the ode snippets to further explain the code."
    ),
    output_type=Tutorial
)

outline_builder  = Agent(
    name="Outline Builder",
    instructions=("Given a programming topic, your job is to create a detailed outline for a tutorial on that topic."
                  "Break down the topic into sections and subsections that logically flow from one another."
                  "Provide a brief description for each section in the outline."
    ),
    handoffs=[tutorial_generator]
)

tutorial_response = await Runner.run(
    outline_builder,
    "Loops in python")

print(tutorial_response.final_output)

Sure! Here is a detailed outline for a tutorial on "Loops in Python":

---

# Tutorial Outline: Loops in Python

---

## 1. Introduction to Loops
   - **Description:** Define what loops are and why they are useful in programming. Briefly explain the concept of iteration and automation.

---

## 2. Types of Loops in Python
   - **Description:** Introduce the two primary loop types in Python: for loops and while loops.

---

## 3. The for Loop
   - **a. Basic Syntax**
     - Description: Explain the basic structure of a for loop in Python.
   - **b. Iterating Over Lists and Strings**
     - Description: Show how for loops can be used to iterate over list and string items.
   - **c. Using range() with for Loops**
     - Description: Introduce the range() function and how it helps with generating sequences for iteration.
   - **d. Nested for Loops**
     - Description: Explain how to use a for loop inside another for loop, and provide practical examples.

---

## 4. The while Loop
   - **a

### Example: Triage Agent with **_handoff()_**

In [9]:
history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
)

math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
)

def on_math_handoff(context: RunContextWrapper[None]):
    print("Handoff to Math Tutor Agent")

def on_history_handoff(context: RunContextWrapper[None]):
    print("Handoff to History Tutor Agent")

triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question",
    handoffs=[handoff(history_tutor_agent, on_handoff=on_history_handoff), 
              handoff(math_tutor_agent, on_handoff=on_math_handoff)]
)

In [10]:
result = await Runner.run(triage_agent, "How do I add 2 and 3 together?")
print(result.final_output)

Handoff to Math Tutor Agent
Let's go through the steps of adding 2 and 3 together:

Step 1: Start with the first number.  
We have 2.

Step 2: Add the second number.  
We need to add 3 to 2.

Step 3: Count up from 2, three times.  
Start at 2, then count:
- 3 (first count)
- 4 (second count)
- 5 (third count)

So, 2 plus 3 equals 5.

Example:
If you have 2 apples and someone gives you 3 more apples, you count all the apples together:
2 apples + 3 apples = 5 apples.

Final Answer:  
2 + 3 = 5


In [ ]:
result = await Runner.run(triage_agent, "In which year did the WW2 end?")
print(result.final_output) 

Handoff to History Tutor Agent
World War II ended in 1945. The conflict came to an official close in Europe on May 8, 1945, known as V-E Day (Victory in Europe Day) when Germany surrendered unconditionally to the Allied forces. The war concluded in the Pacific on September 2, 1945, when Japan formally surrendered following the atomic bombings of Hiroshima and Nagasaki in August 1945. Thus, 1945 marks the end of World War II. If you’d like more context about the events leading up to the end of the war or its aftermath, let me know!


### Example : Customer Service

In [15]:
class ManagerEscalation(BaseModel):
    issue: str # the issue being escalated
    why: str # why can you not handle it? Used for training in the future

@function_tool
def create_ticket(issue: str):
    """ 
    Create a ticket in the system for an issue to be resolved.
    """
    print(f"Creating ticket for issue: {issue}")
    return "Ticket created. ID 123445"
# In a real world scenario, this function would interact with a ticketing system API.

manager_agent = Agent(
    name="Manager", 
    handoff_description="Handles escalated issues that requires managerial attention",
    instructions=(
        "You handle escalated customer issues that the initial custom service agent could not resolve."
        "You will receive the issue the reason for escalatiob. If the issue cannot be immediately resolved for the "
        "customer, create a ticket in the system and inform the customer."
    ),
    tools = [create_ticket],
)

def on_manager_handoff(ctx: RunContextWrapper[None], input: ManagerEscalation):
    print("Handoff to Manager Agent")
    print(f"Issue: {input.issue}")
    print(f"Reason for escalation: {input.why}\n")
    # here we might store the escalation in a datbase or log it for future reference.

customer_service_agent = Agent(
    name= "Customer Service",
    instructions="You assist customers with the general inquiries and basic troubleshooting. "+
    "If the issue cannot be resolved, escalate it to the Manager along with the reason why you cannot fix the issue yourself.",
    handoffs=[handoff(
        agent=manager_agent, 
        input_type=ManagerEscalation, 
        on_handoff=on_manager_handoff)]
)

In [17]:
result = await Runner.run(
    customer_service_agent,
    "I want a refund, but your system won't let me process it. The website is just blank"
    )
print()
print(result.final_output)

Handoff to Manager Agent
Issue: Customer cannot process a refund because the website is blank and not functioning.
Reason for escalation: This issue involves a website malfunction and a financial transaction request, which requires manager authorization and technical support beyond general troubleshooting.

Creating ticket for issue: Customer cannot process a refund due to the website being blank and non-functional. Immediate technical investigation required to restore refund functionality and process the customer's refund.
Creating ticket for issue: Customer requests a refund; requires financial transaction processing once website functionality is restored. Ensure refund request is prioritized and customer updated.

Thank you for your patience. I understand you're unable to process a refund because the website is displaying a blank page. I am escalating this issue to our technical and financial support teams to ensure both the website issue and your refund request are quickly resolved

In [18]:
result = await Runner.run(
    customer_service_agent,
    "Hello, how much does your premium plan cost?"
    )
print()
print(result.final_output)


Our pricing information may vary depending on the service or product you're referring to. Could you please confirm which product or service you’re interested in so I can provide you with the most accurate details about the premium plan cost?
